# 트랜스포머 인코더 구현하기
[Crimsonjoo 깃허브](https://github.com/crimsonjoo/Easy-Transformer)


이 노트북은 PyTorch를 사용하여 트랜스포머 모델의 인코더 부분을 구현하는 방법을 단계별로 설명합니다.

트랜스포머 모델은 주로 자연어 처리(NLP) 분야에서 사용되는 모델로, 여러 개의 인코더와 디코더 레이어로 구성됩니다.

해당 노트북에서는 인코더 부분만을 구현해보겠습니다.


&nbsp;

## 1. 필요한 라이브러리 불러오기

먼저, 이 구현에 필요한 PyTorch 라이브러리를 불러옵니다.

In [ ]:
import torch
import math
from torch import nn
import torch.nn.functional as F

&nbsp;

## 2. 인코더 구성 요소 구현하기

트랜스포머 인코더를 구현하기 전에, 인코더가 사용하는 핵심 구성 요소들을 먼저 구현합니다.

이 구성 요소들은 멀티 헤드 어텐션, 포지션 와이즈 피드포워드 네트워크, 그리고 레이어 노멀라이제이션입니다.


####**- 스케일드 닷 프로덕트 어텐션 함수**

어텐션 메커니즘은 입력 시퀀스의 중요한 부분에 더 많은 주의를 기울이도록 모델을 돕습니다.


스케일드 닷 프로덕트 어텐션은 이러한 어텐션 메커니즘의 한 형태입니다.


In [ ]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]  # 키 벡터의 차원
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)  # 스케일링
    if mask is not None:
        scaled += mask  # 마스크가 있으면 추가
    attention = F.softmax(scaled, dim=-1)  # 소프트맥스를 통한 어텐션 가중치 계산
    values = torch.matmul(attention, v)  # 가중치를 값 벡터에 적용
    return values, attention

####**- 멀티 헤드 어텐션 클래스**

멀티 헤드 어텐션은 모델이 다양한 위치에서 정보를 동시에 얻을 수 있도록 합니다.

이는 모델의 표현력을 향상시키는 데 도움이 됩니다.



In [ ]:
# 멀티 헤드 어텐션 클래스
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model, 3 * d_model)  # Q, K, V를 위한 선형 변환
        self.linear_layer = nn.Linear(d_model, d_model)  # 최종 선형 변환

    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.size()
        qkv = self.qkv_layer(x).view(batch_size, seq_len, self.num_heads, 3 * self.head_dim)
        q, k, v = qkv.chunk(3, dim=-1)
        q, k, v = [tensor.permute(0, 2, 1, 3) for tensor in (q, k, v)]  # 헤드 차원을 앞으로 이동
        values, attention = scaled_dot_product(q, k, v, mask)
        values = values.permute(0, 2, 1, 3).contiguous().view(batch_size, seq_len, -1)
        return self.linear_layer(values)

####**- 레이어 노멀라이제이션 & 포지션 와이즈 피드포워드 네트워크**

레이어 노멀라이제이션은 각 레이어의 입력을 정규화하여 학습을 안정화시키는 기술입니다.

포지션 와이즈 피드포워드 네트워크는 각 위치에서 독립적으로 작동하는 두 개의 선형 변환과 ReLU 활성화 함수로 구성됩니다.



In [ ]:
# 레이어 노멀라이제이션 클래스
class LayerNormalization(nn.Module):
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(features))
        self.beta = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta


# 포지션 와이즈 피드포워드 네트워크 클래스
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.dropout(F.relu(self.linear1(x)))
        return self.linear2(x)

####**- 인코더 레이어 클래스**

인코더 레이어는 멀티 헤드 어텐션과 포지션 와이즈 피드포워드 네트워크를 포함합니다.

이 레이어는 입력 시퀀스를 변환하여 다음 레이어로 전달하는 역할을 합니다.



In [ ]:
# 인코더 레이어 클래스
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.norm1 = LayerNormalization(d_model)
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.norm2 = LayerNormalization(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        x2 = self.norm1(x + self.self_attn(x, mask))
        return self.norm2(x2 + self.ffn(x2))

####**- 인코더 클래스**

이제 모든 구성 요소를 하나로 합쳐 인코더를 구성합니다.

인코더는 여러 개의 인코더 레이어를 포함하며, 각 레이어는 입력 데이터를 처리하여 최종 출력을 생성합니다.



In [ ]:
# 인코더 클래스
class Encoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, num_layers):
        super().__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.norm = LayerNormalization(d_model)

    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

&nbsp;

## 3. 모델 실행

마지막으로, 인코더 모델을 초기화하고 입력 데이터에 대해 실행해보겠습니다.

이를 통해 모델이 정상적으로 작동하는지 확인할 수 있습니다.




In [ ]:
# 모델 config
d_model = 512
num_heads = 8
drop_prob = 0.1
batch_size = 30
max_sequence_length = 200
ffn_hidden = 2048
num_layers = 5


# 모델 선언
encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)


# 모델 실행
x = torch.randn( (batch_size, max_sequence_length, d_model) ) # includes positional encoding
out = encoder(x)